In [1]:
import torch
import torch.nn.functional as F
import random
import math
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad
%matplotlib inline


In [ ]:
# rebuild train dataset

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [7]:
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
    
    print(w)
    context = [0] * block_size # start with padded context of just zero tokens
    #print(context)
    for char in w + '.':
        ix = stoi[char]
        X.append(context)
        Y.append(ix)
        #print(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop context and append
        #print(context)
        
X_ = np.array(X)
Y_ = np.array(Y)
X = torch.tensor(X)
Y = torch.tensor(Y)



emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [8]:
X.shape, X.dtype, Y.shape, Y.dtype


(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [9]:
X_.shape, X_.dtype, Y_.shape, Y_.dtype


((32, 3), dtype('int64'), (32,), dtype('int64'))

In [20]:
C = torch.randn((27, 2))
C_ = np.random.randn(27,2)

In [12]:
C[5]

tensor([ 2.6287, -1.6872])

In [21]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([1.1434, 0.6123])

In [22]:
one_hot = np.eye(27)[5]  # shape (27,)
one_hot@C_


array([ 1.39967093, -1.69281486])

In [ ]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [24]:
C_[X_].shape

(32, 3, 2)

In [27]:
emb = C[X]
emb_ = C_[X_]

In [28]:
np.concatenate([emb_[:, 0, :], emb_[:, 1, :], emb_[:,2, :]], 1).shape

(32, 6)

In [29]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape # dependent on block size


torch.Size([32, 6])

In [30]:
emb.view(emb.shape[0], 6) == torch.cat(torch.unbind(emb, 1), 1)


tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

In [32]:
emb_.reshape(32,6) == np.concatenate([emb_[:,i,:] for i in range(emb_.shape[1])], axis = 1)


array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  T

W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [34]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [36]:
W1_ = np.random.randn(6,100)
b1_ = np.random.randn(100)

In [35]:
(emb.view(-1, 6) @ W1).shape, b1.shape


(torch.Size([32, 100]), torch.Size([100]))

In [42]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h_ = np.tanh(np.reshape(emb_, (-1, 6)) @ W1_ + b1_)



In [38]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)


In [39]:
W2_ = np.random.randn(100,27)
b2_ = np.random.randn(27)

In [43]:
logits = h @ W2 + b2
logits_ = h_ @ W2_ + b2_


In [44]:
counts = logits.exp()
counts_ = np.exp(logits_)

In [46]:
prob = counts / counts.sum(1, keepdim=True)
prob_ = counts_ / counts_.sum(1, keepdims=True)


In [47]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [48]:
Y_

array([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9,
       19,  1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [50]:
prob[torch.arange(32), Y]
prob_[np.arange(32), Y_]

array([2.51917405e-09, 3.18153338e-12, 8.20576510e-13, 4.14908417e-08,
       8.71565217e-01, 2.48141156e-09, 1.18318464e-06, 2.27205561e-16,
       4.04958682e-11, 1.29718592e-11, 2.22859949e-15, 3.52498253e-02,
       3.42856620e-10, 1.27647247e-03, 8.70420274e-11, 1.06920519e-07,
       2.32760325e-10, 1.74694169e-02, 1.35258944e-08, 9.08391768e-01,
       2.21826258e-19, 1.90681028e-10, 2.37528981e-14, 1.08056127e-05,
       1.25471072e-01, 8.75709589e-07, 3.54787014e-08, 9.20529848e-01,
       2.23377917e-04, 3.99019471e-03, 7.84588729e-12, 3.52792724e-02])

In [51]:
prob[torch.arange(32), Y].log().mean()


tensor(-16.4274)

In [52]:
np.log(prob_[np.arange(32), Y_]).mean()


-16.973839136523353

In [37]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[ 0.9033, -0.8239,  0.8567,  ...,  0.3553,  0.6233, -0.0538],
        [-0.7394, -0.7387, -0.5235,  ...,  0.6306,  0.1518, -0.9538],
        [-0.5064, -0.4589, -0.9978,  ...,  0.8674,  0.9948,  0.6432],
        ...,
        [ 0.9128, -0.9998, -0.2513,  ...,  0.8765,  0.0391,  0.6888],
        [ 0.6284,  0.9854,  0.9954,  ..., -0.6096, -0.9994, -0.7153],
        [-0.8970,  0.9991,  0.9931,  ..., -0.3661, -0.9311, -0.9954]])